## **Zadanie 3 - Algorytm min-max z przycinaniem alfa-beta**

Cel zadania polega na implementacji algorytmu min-max z przycinaniem alfa-beta i zastosowaniu go do gry w kółko i krzyżyk (tic-tac-toe). Do rozwiązania zadania można wykorzystać gotową implementację gry w kółko i krzyżyk. Gra ta posiada prosty interfejst tekstowy, zaimplementowanego gracza "ludzkiego" i losowego, jak również "zaślepkę" dla gracza wykorzystującego algorytm min-max.

**Kroki do wykonania:**
- Uruchomienie gry dla gracza losowego i ludzkiego (w celu zapoznania się ze sposobem działania gry).
- Implementacja algorytmu min-max z przycinaniem alfa-beta. Ocena gracza min-max (porównanie jego zachowania w starciu z graczem ludzkim w stosunku do gracza losowego).
- Rozszerzenie gry o możliwość konfiguracji różnych poziomów głębokości drzewa przeszukiwań dla dwóch graczy min-max. Uruchomienie gry dla dwóch graczy min-max o różnej głębokości drzewa przeszukiwań (dla kilku wartości tego parametru) i skomentowanie wpływu głębokości przeszukiwania drzewa gry na jakość wyników uzyskiwanych przez graczy.

**Uwaga**: Punkty 2 i 3 proszę wykonać dla standardowego rozmiaru planszy (3x3), jak również dla jednego wybranego większego rozmiaru.

In [ ]:
import random
from typing import List, Tuple, Optional

In [ ]:
class Board:
    def __init__(self, size: int = 3) -> None:
        if size <= 0:
            raise ValueError("Size must be greater than 0")

        self.size = size
        self.board = [" " for _ in range(size ** 2)]
        self.player_circle = "O"
        self.player_cross = "X"
        self.current_player_idx = 0

    def print_board(self) -> None:
        for i in range(self.size):
            print(" ---" * self.size)
            print("| " + " | ".join(self.board[i * self.size: (i + 1) * self.size]) + " |")
        print(" ---" * self.size)

    def get_empty_positions(self) -> List[int]:
        return [i for i, x in enumerate(self.board) if x == " "]

    def is_full(self) -> bool:
        return " " not in self.board

    def get_winner(self) -> Optional[str]:
        # check rows
        for i in range(self.size):
            row = self.board[i * self.size: (i + 1) * self.size]
            if row[0] != " " and all(x == row[0] for x in row):
                return row[0]

        # check columns
        for i in range(self.size):
            col = [self.board[j * self.size + i] for j in range(self.size)]
            if col[0] != " " and all(x == col[0] for x in col):
                return col[0]

        # check diagonals
        diag1 = [self.board[j * self.size + j] for j in range(self.size)]
        diag2 = [self.board[j * self.size + self.size - j - 1] for j in range(self.size)]
        if diag1[0] != " " and all(x == diag1[0] for x in diag1):
            return diag1[0]
        if diag2[0] != " " and all(x == diag2[0] for x in diag2):
            return diag2[0]

        return None

    def play(self, position: int) -> None:
        if position < 0 or position >= self.size ** 2:
            raise ValueError("Invalid position")

        if self.board[position] != " ":
            raise ValueError("Position already taken")

        self.board[position] = self.player_circle if self.current_player_idx == 0 else self.player_cross
        self.current_player_idx = 1 - self.current_player_idx

    def undo(self, position: int) -> None:
        if position < 0 or position >= self.size ** 2:
            raise ValueError("Invalid position")

        if self.board[position] == " ":
            raise ValueError("Position is empty")

        self.board[position] = " "
        self.current_player_idx = 1 - self.current_player_idx

In [ ]:
class Player:
    def __init__(self, name: str) -> None:
        self.name = name

    def play(self, board: Board) -> int:
        raise NotImplementedError

In [ ]:
class RandomPlayer(Player):
    def __init__(self, name: str) -> None:
        super().__init__(name)

    def play(self, board: Board, side: str) -> int:
        empty_positions = board.get_empty_positions()
        if not empty_positions:
            raise ValueError("No empty positions")

        return random.choice(empty_positions)

In [ ]:
class HumanPlayer(Player):
    def __init__(self, name: str) -> None:
        super().__init__(name)

    def play(self, board: Board, side: str) -> int:
        empty_positions = board.get_empty_positions()
        while True:
            try:
                position = int(input(f"{self.name}, enter your move (0-{board.size ** 2 - 1}): "))
                if position in empty_positions:
                    return position
                else:
                    print("Invalid move")
            except ValueError:
                print("Invalid input")

In [ ]:
class MiniMaxPlayer(Player):
    def __init__(self, name: str, max_depth: int = 9) -> None:
        super().__init__(name)
        self.max_depth = max_depth
        self.heuristics = None

    def play(self, board: Board, side: str) -> int:
        if self.heuristics is None:
            self.heuristics = self.get_heuristic_matrix(board)
        _, best_move = self.find_best_move(board, side)
        return best_move

    def minimax(
        self,
        board: Board,
        side: str,
        depth: int,
        is_maximizing: bool,
        alpha: float = float("-inf"),
        beta: float = float("inf")
    ) -> float:
        if depth == 0 or board.get_winner() is not None or board.is_full():
            return self.evaluate(board, side)

        if is_maximizing:
            for move in board.get_empty_positions():
                board.play(move)
                alpha = max(alpha, self.minimax(board, side, depth - 1, not is_maximizing, alpha, beta))
                board.undo(move)
                if beta <= alpha:
                    return alpha
            return alpha
        else:
            for move in board.get_empty_positions():
                board.play(move)
                beta = min(beta, self.minimax(board, side, depth - 1, not is_maximizing, alpha, beta))
                board.undo(move)
                if beta <= alpha:
                    return beta
            return beta

    def find_best_move(self, board: Board, side: str) -> Tuple[float, int]:
        best_score = float("-inf")
        best_move = None

        for move in board.get_empty_positions():
            board.play(move)
            score = self.minimax(board, side, self.max_depth, False)
            board.undo(move)

            if score > best_score:
                best_score = score
                best_move = move

        return best_score, best_move

    def get_heuristic_matrix(self, board: Board) -> List[int]:
        scores = []
        for row in range(board.size):
            for col in range(board.size):
                score = 2 # one for row, one for col
                if row == col:
                    score += 1
                if row + col == board.size - 1:
                    score += 1
                scores.append(score)

        return scores

    def heuristic_function(self, board: Board, side: str) -> int:
        result = 0
        for i in range(board.size * board.size):
            if board.board[i] == side:
                result += self.heuristics[i]
            elif board.board[i] != " ":
                result -= self.heuristics[i]

        return result

    def evaluate(self, board: Board, side: str) -> int:
        if board.get_winner() == side:
            return 100
        elif board.get_winner() != side and board.get_winner() is not None:
            return -100
        elif board.is_full():
            return 0
        else:
            return self.heuristic_function(board, side)

In [ ]:
class Game:
    def __init__(
        self,
        board_size: int = 3,
        players: list[Player] = [RandomPlayer, MiniMaxPlayer]
    ) -> None:
        self.board = Board(board_size)
        self.players = players

    def run_game(self) -> None:
        turn_idx = 0
        while self.board.get_winner() is None and not self.board.is_full():
            player = self.players[turn_idx % 2]
            player_sign = "X" if turn_idx % 2 == 0 else "O"
            move = player.play(self.board, player_sign)
            self.board.play(move)
            print(f"{player.name} PLAYED {move}:")
            self.board.print_board()
            turn_idx += 1

        print(f"\nGAME OVER! {self.board.get_winner()} WINS!" if self.board.get_winner() is not None else "GAME OVER! IT'S A DRAW!")
        self.board.print_board()

In [ ]:
player_1 = MiniMaxPlayer("X", 1)
player_2 = RandomPlayer("O")

In [ ]:
game = Game(4, [player_1, player_2])

In [ ]:
game.run_game()